# Library

In [1]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, random
import pandas as pd
import numpy as np

from selenium import webdriver
import chromedriver_autoinstaller as ca

from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## chrome driver 설치

In [2]:
# USB error 메세지 발생 해결을 위한 코드
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

In [3]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'131'

In [4]:
# # 크롬 드라이버 확인 및 설치(처음 한번만 실행)
# ca.install(True)

# 페이지 접속

In [5]:
url = 'https://finance.naver.com/' # 네이버 증권 > 홈

## requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [7]:
req = requests.get(url)
print(req)

# html = req.content.decode('utf-8') # 한글 깨짐 해결 코드
html = req.text
soup = bs(html, 'html.parser')

soup.title.text

<Response [200]>


'네이버페이 증권'

## selenium 작동

In [ ]:
# 웹드라이버 실행
driver = webdriver.Chrome(options=options)

In [ ]:
driver.get(url)
driver.implicitly_wait(10) # 페이지 로드 될 때까지 기다리지만 로드 되는 순간 종료
# driver.maximize_window() # 브라우져 창 최대화

# 정보 수집

In [9]:
# TOP 종목
section_sise_top = soup.select_one('div.section_sise_top')
section_sise_top

<div class="section_sise_top">
<h2 class="h_type"><span>TOP 종목</span></h2>
<ul class="tab_area sise_top1">
<li class="tab1"><a class="is_active" href="#" onclick="clickcr(this, 'spe.sale', '', '', event);return false;"><span>거래상위</span></a></li>
<li class="tab2"><a href="#" onclick="clickcr(this, 'spe.ceiling', '', '', event);return false;"><span>상승</span></a></li>
<li class="tab3"><a href="#" onclick="clickcr(this, 'spe.bottom', '', '', event);return false;"><span>하락</span></a></li>
<li class="tab4"><a href="#" onclick="clickcr(this, 'spe.value', '', '', event);return false;"><span>시가총액 상위</span></a></li>
</ul>
<div class="group_type is_active">
<table class="tbl_home">
<caption>거래상위 TOP 종목<span>종목명에 대한 현재가, 전일대비, 등락률로 구분되어 있습니다.</span></caption>
<colgroup>
<col width="40%"/>
<col/>
</colgroup>
<thead>
<tr>
<th scope="col">종목명</th>
<th scope="col">현재가</th>
<th scope="col">전일대비</th>
<th scope="col">등락률</th>
</tr>
</thead>
<tbody id="_topItems1">
<tr class="up">
<th scope="row"><a href=

In [17]:
group_type_is_active = section_sise_top.select_one('div.group_type.is_active')
group_type_is_active

<div class="group_type is_active">
<table class="tbl_home">
<caption>거래상위 TOP 종목<span>종목명에 대한 현재가, 전일대비, 등락률로 구분되어 있습니다.</span></caption>
<colgroup>
<col width="40%"/>
<col/>
</colgroup>
<thead>
<tr>
<th scope="col">종목명</th>
<th scope="col">현재가</th>
<th scope="col">전일대비</th>
<th scope="col">등락률</th>
</tr>
</thead>
<tbody id="_topItems1">
<tr class="up">
<th scope="row"><a href="/item/main.naver?code=252670" onclick="clickcr(this, 'spe.slist', '252670', '1', event);">KODEX 200선물인버스2X</a></th>
<td>2,605</td>
<td><em class="bu_p bu_pup"><span class="blind">상승</span></em> 60</td>
<td>
<em class="up">
				+2.36%
				</em>
</td>
</tr>
<tr class="down">
<th scope="row"><a href="/item/main.naver?code=900280" onclick="clickcr(this, 'spe.slist', '900280', '2', event);">골든센츄리</a></th>
<td>9</td>
<td><em class="bu_p bu_pdn"><span class="blind">하락</span></em> 3</td>
<td>
<em class="down">
				-25.00%
				</em>
</td>
</tr>
<tr class="up">
<th scope="row"><a href="/item/main.naver?code=050890" onclic

In [26]:
stock_lines = group_type_is_active.select('tbody#_topItems1 tr')
stock_lines

[<tr class="up">
 <th scope="row"><a href="/item/main.naver?code=252670" onclick="clickcr(this, 'spe.slist', '252670', '1', event);">KODEX 200선물인버스2X</a></th>
 <td>2,605</td>
 <td><em class="bu_p bu_pup"><span class="blind">상승</span></em> 60</td>
 <td>
 <em class="up">
 				+2.36%
 				</em>
 </td>
 </tr>,
 <tr class="down">
 <th scope="row"><a href="/item/main.naver?code=900280" onclick="clickcr(this, 'spe.slist', '900280', '2', event);">골든센츄리</a></th>
 <td>9</td>
 <td><em class="bu_p bu_pdn"><span class="blind">하락</span></em> 3</td>
 <td>
 <em class="down">
 				-25.00%
 				</em>
 </td>
 </tr>,
 <tr class="up">
 <th scope="row"><a href="/item/main.naver?code=050890" onclick="clickcr(this, 'spe.slist', '050890', '3', event);">쏠리드</a></th>
 <td>6,630</td>
 <td><em class="bu_p bu_pup"><span class="blind">상승</span></em> 250</td>
 <td>
 <em class="up">
 				+3.92%
 				</em>
 </td>
 </tr>,
 <tr class="up">
 <th scope="row"><a href="/item/main.naver?code=016600" onclick="clickcr(this, 'sp

In [27]:
# 1번째 종목
one_stock = stock_lines[0]
one_stock

<tr class="up">
<th scope="row"><a href="/item/main.naver?code=252670" onclick="clickcr(this, 'spe.slist', '252670', '1', event);">KODEX 200선물인버스2X</a></th>
<td>2,605</td>
<td><em class="bu_p bu_pup"><span class="blind">상승</span></em> 60</td>
<td>
<em class="up">
				+2.36%
				</em>
</td>
</tr>

In [28]:
# 종목명
stock_name = one_stock.select_one('th a').text
stock_name

'KODEX 200선물인버스2X'

In [30]:
# 현재가
current_price = int(one_stock.select_one('td').text.replace(',', ''))
current_price

2605

In [ ]:
# 전일대비

compare_yesterday = one_stock.select('td')[1].text
compare_yesterday

'상승 60'

In [ ]:
# 등락률